#**Deep Learning homework**

In [1]:
import os
import shutil
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# Remove old zip if exists
if os.path.exists("data.zip"):
    os.remove("data.zip")

# Download new data.zip
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip -O data.zip

# Remove existing data/ folder
if os.path.exists("data"):
    shutil.rmtree("data")

# Unzip the new dataset
!unzip -q data.zip

data_dir = "data"
train_dir = os.path.join(data_dir, "train")
test_dir  = os.path.join(data_dir, "test")

--2025-11-19 09:56:52--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-19T10%3A49%3A58Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-19T09%3A49%3A00Z&ske=2025-11-19T10%3A49%3A58Z&sks=b&skv=2018-11-09&sig=nhHFkIBprsW4Tsaul8EMPatCPupsqzMPI8Rztag%2Fa%2BU%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzU0ODAxMiwibmJmIjoxNzYzNTQ2MjEyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [4]:
# Resize to 200x200, convert to tensor
train_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

In [5]:
# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

In [6]:
# DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['curly', 'straight']


In [7]:
model = nn.Sequential(
    # Conv2d: in_channels=3, out=32, kernel=3x3
    nn.Conv2d(
        in_channels=3,
        out_channels=32,
        kernel_size=3
    ),
    nn.ReLU(),

    # MaxPool2d: 2x2
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Flatten
    nn.Flatten(),

    # Fully-connected 64 units + ReLU
    nn.Linear(32 * 99 * 99, 64),
    nn.ReLU(),

    # Output layer: 1 neuron
    nn.Linear(64, 1),
)

print(model)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=313632, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1, bias=True)
)


In [8]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)

###**<font color='red'>Question 1</font>**
Which loss function you will use?
- nn.MSELoss()
- <font color='green'>nn.BCEWithLogitsLoss()</font> ✅
- nn.CrossEntropyLoss()
- nn.CosineEmbeddingLoss()

(Multiple answered can be correct, so pick any)

In [9]:
criterion = nn.BCEWithLogitsLoss()